In [1]:
class GridWorld:
    def __init__(self):
        # Initialize grid dimensions
        self.states = None
        self.rows = 3
        self.cols = 4
        self.gamma = 0.9  # Discount factorr
        self.actions = ['N', 'S', 'E', 'W']
        self.obstacle_position = (2, 2)

        self.initialize_states()

        # initialize transition probabilities
        self.probs = {
            'N': [(0.8, (-1, 0)), (0.1, (0, 1)), (0.1, (0, -1))],
            'S': [(0.8, (1, 0)), (0.1, (0, 1)), (0.1, (0, -1))],
            'E': [(0.8, (0, 1)), (0.1, (-1, 0)), (0.1, (1, 0))],
            'W': [(0.8, (0, -1)), (0.1, (-1, 0)), (0.1, (1, 0))]
        }
    
    def initialize_states(self):
        """Generates a list of grid positions, excluding obstacle position."""
        self.states = [(i + 1, j + 1) for i in range(self.rows)
                       for j in range(self.cols)
                       if (i + 1, j + 1) != self.obstacle_position]

    def get_reward(self, state):
        """returns reward for given state."""
        if state == (4, 3):
            return 1.0
        elif state == (4, 2):
            return -1.0
        else:
            return -0.02

    def is_valid_state(self, state):
        """check if state is valid (within bounds and not obstacle)."""
        x, y = state
        return 1 <= x <= self.cols and 1 <= y <= self.rows and state != (2, 2)

    def get_next_state(self, current_state, action_effect):
        """get next state given current state and action effect."""
        next_x = current_state[0] + action_effect[1]
        next_y = current_state[1] + action_effect[0]
        next_state = (next_x, next_y)

        # if next state is invalid, stay in current state
        if not self.is_valid_state(next_state):
            return current_state

        #if next state is valid, move there
        return next_state



In [2]:
def value_iteration(grid, epsilon=1e-6, max_iterations=1000):
    """
    Implement value iteration algorithm.
    Returns optimal value function and policy.
    """
    # Initialize value function
    V = {state: 0 for state in grid.states}
    # Set terminal state values
    V[(4, 3)] = 1.0  # Goal state
    V[(4, 2)] = -1.0  # Pit state

    policy = {state: None for state in grid.states}

    iteration = 0
    while iteration < max_iterations:
        delta = 0
        V_old = V.copy()

        # Update each state
        for state in grid.states:
            # Skip terminal states
            if state in [(4, 3), (4, 2)]:
                continue

            # Find maximum value over all actions
            action_values = []

            for action in grid.actions:
                value = 0
                # Calculate expected value for each possible outcome
                for prob, effect in grid.probs[action]:
                    next_state = grid.get_next_state(state, effect)
                    # Make sure next_state is in our state space
                    if next_state in V_old:
                        value += prob * (grid.get_reward(state) + grid.gamma * V_old[next_state])
                action_values.append((value, action))

            # Update value and policy
            best_value, best_action = max(action_values)
            V[state] = best_value
            policy[state] = best_action

            delta = max(delta, abs(V[state] - V_old[state]))

        if delta < epsilon:
            break

        iteration += 1

    return V, policy


In [3]:
def print_results(V, policy):
    """Print the results in a grid format."""
    print("\nOptimal Value Function:")
    for y in range(1, 4):
        for x in range(1, 5):
            if (x, y) == (2, 2):
                print("  ###  ", end="")
            elif (x, y) in V:
                print(f"{V[(x, y)]:7.3f}", end="")
        print()

    print("\nOptimal Policy:")
    for y in range(1, 4):
        for x in range(1, 5):
            if (x, y) == (2, 2):
                print("  #  ", end="")
            elif (x, y) in policy:
                if (x, y) in [(4, 3), (4, 2)]:
                    print("  *  ", end="")
                else:
                    print(f"  {policy[(x, y)]}  ", end="")
        print()


In [4]:
# Run the algorithm
grid = GridWorld()
V, policy = value_iteration(grid)
print_results(V, policy)


Optimal Value Function:
  0.391  0.320  0.392
  0.482  ###    0.529 -1.000
  0.577  0.697  0.822  1.000

Optimal Policy:
  S    E    S  
  S    #    S  
  E    E    E  
